### Relatório Dataming - Avito Demand Prediction Challenge - Parte 4 - Predições com Regressão Logística e Linear

#### Time:
- Filipe Koop
- Lucas Pérez
- Raphael Coelho

In [2]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
dir = 'c:/_dados/_avito/'

Para a última abordagem, optou-se por utilizar dois modelos de machine learning: regressão linear e regressão logística. Para isso foi utilizado a biblioteca sklearn juntamente com os dois modelos. Para tal aplicação, foi necessário um pré processamento na base. As variáveis alvo eram user_type e parent_category_name juntamento com o deal_probability. Entretanto, o modelo de regressão Logística só aceita entradas binárias, e para isso, essas três colunas foram desmembradas.

A partir disso, passamos a ter 10 colunas binárias ao invès do deal_probability contínuo, e mais doze colunas, 9 de parent_category_name e 3 de user_type, todas binárias ao invès das suas versões contínuas.

In [49]:
df = pd.read_pickle(dir + 'df2.pickle')
df.fillna(0, inplace=True)
df = df.reset_index()
df['deal_chance'] = (df.deal_probability > 0.6) *1
df.head()

,index,activation_date,category_name,city,deal_probability,deal_rounded,description,image_top_1,item_id,month,...,sold_1,sold_2,sold_3,sold_4,sold_5,sold_6,sold_7,sold_8,sold_9,deal_chance
0,0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,0.2,"Кокон для сна малыша,пользовались меньше месяц...",1008.0,b912c3c6a6ad,3,...,0,0,0,0,0,1,0,0,0,0
1,1,2017-03-26,Мебель и интерьер,Самара,0.00000,0.0,"Стойка для одежды, под вешалки. С бутика.",692.0,2dac0150717d,3,...,0,0,0,0,0,0,0,0,0,0
2,2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,0.4,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0,ba83aefab5dc,3,...,0,0,0,0,0,1,0,1,1,0
3,3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,0.8,Продам кресло от0-25кг,796.0,02996f1dd2ea,3,...,1,1,1,1,1,0,0,1,1,1
4,4,2017-03-16,Автомобили,Волгоград,0.20797,0.2,Все вопросы по телефону.,2264.0,7c90be56d2ab,3,...,0,0,0,0,1,0,0,0,0,0


In [50]:
groups = df.parent_category_name.unique()
df['cat']=''
for i,group in enumerate(groups):
    df['cat_' + str(i)] = (df.parent_category_name == group) * 1
user_type = pd.get_dummies(df.user_type)
df = df.join(user_type)
df.columns

Index(['index', 'activation_date', 'category_name', 'city', 'deal_probability',
       'deal_rounded', 'description', 'image_top_1', 'item_id', 'month',
       'param_1', 'param_2', 'param_3', 'parent_category_name', 'price',
       'region', 'source', 'title', 'user_id', 'user_type', 'price_normal_log',
       'price_normal', 'price_top', 'sold_0', 'sold_1', 'sold_2', 'sold_3',
       'sold_4', 'sold_5', 'sold_6', 'sold_7', 'sold_8', 'sold_9',
       'deal_chance', 'cat', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'cat_4',
       'cat_5', 'cat_6', 'cat_7', 'cat_8', 'Company', 'Private', 'Shop'],
      dtype='object')

In [14]:
df.head()

,index,activation_date,category_name,city,deal_probability,deal_rounded,description,image_top_1,item_id,month,...,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,Company,Private,Shop
0,0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,0.2,"Кокон для сна малыша,пользовались меньше месяц...",1008.0,b912c3c6a6ad,3,...,0,0,0,0,0,0,0,0,1,0
1,1,2017-03-26,Мебель и интерьер,Самара,0.00000,0.0,"Стойка для одежды, под вешалки. С бутика.",692.0,2dac0150717d,3,...,0,0,0,0,0,0,0,0,1,0
2,2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,0.4,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0,ba83aefab5dc,3,...,1,0,0,0,0,0,0,0,1,0
3,3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,0.8,Продам кресло от0-25кг,796.0,02996f1dd2ea,3,...,0,0,0,0,0,0,0,1,0,0
4,4,2017-03-16,Автомобили,Волгоград,0.20797,0.2,Все вопросы по телефону.,2264.0,7c90be56d2ab,3,...,0,1,0,0,0,0,0,0,1,0


### 3.4- Regressão Logística

Com a base já montada, foi feito dois testes iniciais com regressão logística, onde as variáveis de train receberam tudo da base 'train' e a variável de teste, tudo da base 'test'. Organizou-se as variáveis de predição, target, test e label e foi dado inicio ao modelo. É importante mencionar que a variável 'y_pred' nos dá ambas as probabilidades, de não vender [0] e de vender, [1]. Entretanto, o objetivo do teste é focado na probabilidade de venda. Ao final, o resultado foi gravado em um arquivo txt para submissão.

Na saída, temos primeiramente a taxa de acerto do modelo, que na primeira tentativa chegou-se a 86% e em seguida a média da variável alvo, a média da variavél de predição e todos os valores preditos.

Para o segunda teste, foi seguido o mesmo modelo, com o mesmo protocolo de escrita do código, mudando apenas o parametro C da regressão Logística, cujo qual se refere a força de regularização do modelo onde valores menores especificam regularização mais fortes. Com isso, a taxa de acerto da segunda tentativa subiu para quase 89%.

In [27]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['sold_0'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LogisticRegression(C=5.0, random_state=0)
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict_proba(test)[:,1]

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00001.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\r\n")
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\r\n")
        
#### SCORE 0.2510

0.8608369960836065

0.13915834787791068
0.1437716476155714
[0.0794548  0.14782958 0.18697061 ... 0.08002606 0.40982857 0.07581857]


In [28]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['deal_chance'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LogisticRegression(C=1.0, random_state=0)
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict_proba(test)[:,1]

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00002.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\r\n")
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\r\n")
        
#### SCORE 0.2556

0.8878353678004343

0.11216130645779235
0.11482288390438741
[0.07276252 0.13173594 0.16779363 ... 0.07365233 0.24878448 0.04735115]


Para o terceiro teste, foi executado o modelo 10 vezes para as colunas de sold, que como já foi dito anteriormente, se refere a coluna de deal_probability mas em binário. Logo, para cada uma dessas colunas foi executado a regressão logística mantendo o valor de C=1.0.

Foi observado então, que todas as colunas tiveram um valor aproximado em 86%, mesmo com o valor de C reduzido. Apesar disso, nas submissões, foi observado que este terceiro teste teve um score bem parecido com o primeiro teste, com uma diferença de apenas 0.0001, e mesmo caindo 2% na taxa de acurácia, em relação ao teste anterior, foi melhor em score de submissão.

In [30]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values
predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values

result = []
for g in range(10):
    target = X_train['sold_' + str(g)].values
    model  = LogisticRegression(C=1.0, random_state=0)
    model.fit(predictors, target)
    display(model.score(predictors,target))
    y_pred = model.predict_proba(test)[:,1]
    result.append(y_pred)

y_pred2 = []
for i in range(len(y_pred)):
    f = 0
    for g in range(10):
        f +=  result[g][i]/10
    y_pred2.append(f)
    
print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00003.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\n")
    for label_, predict_ in zip(label, y_pred2):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\n")
        
#### SCORE 0.2511

0.8608369960836065

0.8605915563407263

0.8609114926993316

0.8607944265889064

0.8606673832531608

...

0.8610890873100336

0.8605489868460261

0.8605157294282917

0.8608137158911924

0.1391809629219701
0.14382483593094558
[0.07939672 0.14674548 0.18713147 ... 0.07996054 0.41175167 0.07421595]


### 3.5- Regressão Linear

Por fim, testou-se a ultima abordagem: regressão linear. A estratégia de montagem do algortimo foi a mesma dos testes anteriores. As diferenças foram apenas no modelo em si, e na utilização do deal_probabbility como target. 

Podemos ver então, que o resultado da acurácia ficou muito baixo, por volta 11%. Isso pode nos mostrar que o modelo previu muito valores 

In [37]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['deal_probability'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LinearRegression()
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict(test)

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00004.csv', 'w') as f:
    f.write('item_id,deal_probability\n')
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4).clip(0,0.99))+ '\n')
        
#### SCORE 0.2513

0.11143490760183972

0.13913057577902177
0.13991851371100106
[0.08270784 0.15046021 0.1871146  ... 0.08344773 0.40809342 0.09394676]


### 4. Conclusão

Pela observação dos aspectos analisados constatou-se diversos resultados relevantes para o tratamento do problema em questão. Dentro do escopo das análises estatísticas, foi possível perceber diversos padrões de valores, tempo, e perfil de usuário que podem ser levados em consideração na hora de um anunciante ofertar o produto na plataforma Avito.

Na análise textual feita em cima das descrições dos produtos, não foi constatado um padrão de termos ou palavras, umas vez que as grandes categorias de anúncios podem ser completamente distintas umas das outras.

Já na análise com os modelos de machine learning, foi possivel observar que dentre as três abordagem usadas, (naive bayes, regressão logística e regressão linear) a que mais se sobressaiu foi a de regressão logística, com melhores resultados de acurácia.

Analisando todas as abordagens em conjunto, foi possivel entender a essência do problema e dos dados e correlacionar as variáveis pertinentes utilizand tudo o que foi mostrado neste trabalho podendo assim dar uma resposta mais precisa sobre a probabilidade de compra ou não de um produto. E estes modelos, juntamento com as analises feitas em cima das categorias, preços e usuários, permitem que os anunciantes possam traçar uma estratégia melhor, um publico melhor, uma descrição melhor para os seus anúncios.